ok so im not 100% certain if it will work bc iv never tested with someone else's acct so lmk.

steps are:

1) type in the dates u want (the search playlist id is alr set to ihy skai)

2) run the first cell (play button is top left of each cell). it should open up a new tab

3) log into spotify and then copy the url of that new tab (should be like http://localhost:7777 etc.)

4) run the second cell and paste the url into the prompt box that shows up

5) run the third cell

hopefully theres no errors and it'll create a new playlist called "era//date1 : date2"

there is a limit to how many times you can call the spotify api and i didnt have the energy to handle that, so if it is throwing errors just try waiting a couple minutes and then starting again


In [140]:
################################################################################

start_date = '2020-10-01' #yyyy-mm-dd
end_date = '2021-10-25'
searchspace_playlist_id = '6y8vuKAhLZtk2l4UOzrg13'

################################################################################

import pandas as pd

#for apis
import requests
from urllib.parse import urlencode
import json
import base64
import webbrowser

#for working around colab not being able to open windows
from IPython.display import Javascript

#spotify keys
client_id = '9bf2211d3826492aa72471ec394689e1'
client_secret = '8e28cf71dcc5438eb06e5da566a3ec28'

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "playlist-modify-private"}

def open_web():
    url = "https://accounts.spotify.com/authorize?" + urlencode(auth_headers)
    display(Javascript('window.open("{url}");'.format(url=url)))

open_web()

<IPython.core.display.Javascript object>

In [141]:
full_callback = input("enter url: ")
code = full_callback.split("code=")[1]

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)

print(r.json())
token = r.json()["access_token"]

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json"
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

enter url: http://localhost:7777/callback?code=AQD2ORyirCUBMvDMUFbUbHmTjZvrvHlaN_VfkFn-yt2h2PNZULbFCeIepkhUuZ2SzKzfig8xfK7lkZSW3LYJCUVwyW8lsWdxE3bXxTdB1i1LzZmlj3NNoPn6fayQYKvk28MGtlapLGFW9kNn3uVGxeUeEkgridTXnWs5Y7CNWTFszn_kxp-DnqEObrckT_O0cohyBJvkfDybQW4
{'access_token': 'BQCAUVXsaS5SSZ6PI3EoNuipoL_6X1YFx45ckRWYVJdRUReGDY6wRvqqzasbWhb95KlhYT_atzaXmp6Qeo0TKNmS_adnNdqogfGezcEbhNOhHsLUyCJCq_cb5MzErs0p_is0YE7wmBY_h5cPUBkglQ4EJFOmVnl1ve-QQjJOJnp8MXnevCShBlKk-372IUo551bGw1sdK1Td4kAa', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQCvoIGSMj-Q_6fBlqNVIRhJD1kYyc7oT0ySAAmp6nmO1NjoDNrBw2gf7jasTUhX42rKjevFiQ5uQ-vaB5oXUhLcVRbOvsa4ahMl74GHGX7d6kxf5SxOI8x0oFHD4dbBj9Y', 'scope': 'playlist-modify-private'}


In [142]:
def subset_dict_by_date_range(dictionary, start_date, end_date):
    filtered_keys = [key for key in dictionary.keys() if start_date <= key <= end_date]
    subset_dict = {key: dictionary[key] for key in filtered_keys}
    return subset_dict

##pull songs for search space
songs = {}
offset = 0
while True:
  playlist_endpoint = 'playlists/{}/tracks?limit=50&offset={}'.format(searchspace_playlist_id, offset)
  playlist_request = requests.get(BASE_URL + playlist_endpoint, 
                  headers=headers)
  playlist_json = playlist_request.json()

  for i in range(len(playlist_json['items'])):

    track_id = "spotify:track:" + playlist_json['items'][i]['track']['id']
    added_date = playlist_json['items'][i]['added_at'].split('T')[0]

    if added_date in songs:
      songs[added_date].append(track_id)
    else:
      songs[added_date] = [track_id]
    
    offset += 1

  if offset%50 != 0:
    break

target = subset_dict_by_date_range(songs, start_date, end_date)
uris = [item for sublist in target.values() for item in sublist]

#create new playlist
get_user_endpoint = 'me'
get_user_request = requests.get(BASE_URL + get_user_endpoint, 
                  headers=headers)
get_user_json = get_user_request.json()
user_id = get_user_json['id']

create_body = {"name": "era//{} : {}".format(start_date[:7], end_date[:7]),"public": "false"}

create_playlist_endpoint = 'users/{}/playlists'.format(user_id)
create_playlist_request = requests.post(BASE_URL + create_playlist_endpoint, 
                  headers=headers, json = create_body)
create_playlist_json = create_playlist_request.json()

#add to playlist
while uris:
  req_uris = uris[:100]
  uris = uris[100:]

  add_uri_string = ','.join(str(item) for item in req_uris)

  add_playlist_endpoint = 'playlists/{}/tracks?uris={}'.format(create_playlist_json['id'], add_uri_string)
  add_playlist_request = requests.post(BASE_URL + add_playlist_endpoint, 
                  headers=headers)
  add_playlist_json = add_playlist_request.json()